In [1]:
from fbprophet.diagnostics import cross_validation, performance_metrics
import itertools

road = '10'

new_df = pd.DataFrame({
    'ds':new_train['날짜'],
    'y':new_train[road]
})

In [2]:
import os
import warnings

import pandas as pd
from datetime import datetime

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 37)
warnings.filterwarnings(action='ignore')

In [3]:
DATA_DIR = os.path.abspath(os.path.dirname('EDA.ipynb') + 'data' + '/EDA')

In [4]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train', 'train.csv'))
valid_df = pd.read_csv(os.path.join(DATA_DIR, 'train', 'validate.csv'))

In [5]:
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test', 'test.csv'))
submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

In [6]:
def convert(date):
    return date[:4] + '-' + date[4:6] + '-' + date[6:]

def date_to_timestamp(df):
    new_df = df.copy()
    
    date_series = new_df['날짜'].astype(str).map(convert)
    time_series = new_df['시간'].astype(str).map(lambda x: ' ' + x.zfill(2) + ':00:00')
    
    new_df['날짜'] = date_series + time_series
    
    new_df['날짜'] = pd.to_datetime(new_df['날짜'])
    new_df.drop(['시간'], axis=1, inplace=True)
    
    return new_df

In [7]:
train = date_to_timestamp(train_df)
train

                      날짜        10       100      101      120      121  \
0    2020-01-01 00:00:00   83247.0   19128.0   2611.0   5161.0   1588.0   
1    2020-01-01 01:00:00   89309.0   19027.0   3337.0   5502.0   1650.0   
2    2020-01-01 02:00:00   66611.0   14710.0   2970.0   4631.0   1044.0   
3    2020-01-01 03:00:00   53290.0   13753.0   2270.0   4242.0   1021.0   
4    2020-01-01 04:00:00   52095.0   17615.0   2406.0   3689.0   1840.0   
...                  ...       ...       ...      ...      ...      ...   
3307 2020-05-17 19:00:00  311727.0  101285.0  10085.0  30637.0  10060.0   
3308 2020-05-17 20:00:00  305354.0   91426.0   8607.0  26021.0   8095.0   
3309 2020-05-17 21:00:00  306008.0   75113.0   6325.0  19933.0   5711.0   
3310 2020-05-17 22:00:00  237447.0   49498.0   4209.0  12145.0   3891.0   
3311 2020-05-17 23:00:00  150312.0   27410.0   2350.0   6406.0   1803.0   

         140       150     160      200     201      251      270      300  \
0      892.0   32263.

In [8]:
valid = date_to_timestamp(valid_df)
valid[168:]

                     날짜      10      100      101      120     121     140  \
168 2020-05-18 00:00:00   82065  15172.0   1500.0   3294.0  1086.0   962.0   
169 2020-05-18 01:00:00   51248   9840.0    813.0   2356.0   696.0   546.0   
170 2020-05-18 02:00:00   39026   7894.0    760.0   2413.0   408.0   549.0   
171 2020-05-18 03:00:00   40993  10137.0    780.0   2701.0   420.0   741.0   
172 2020-05-18 04:00:00   77863  19603.0   1276.0   5019.0   968.0  1160.0   
..                  ...     ...      ...      ...      ...     ...     ...   
331 2020-05-24 19:00:00  314226  98345.0  10625.0  28618.0  8316.0  6684.0   
332 2020-05-24 20:00:00  300001  87871.0   8226.0  22706.0  6981.0  5743.0   
333 2020-05-24 21:00:00  304150  71126.0   6002.0  18317.0  4939.0  3779.0   
334 2020-05-24 22:00:00  236751  44947.0   3575.0  11455.0  3135.0  2536.0   
335 2020-05-24 23:00:00  143609  26137.0   2242.0   6166.0  1609.0  1391.0   

        150     160    200     201      251      270      300  

In [9]:
test = date_to_timestamp(test_df)
test.head()

                   날짜     10    100     101   120     121     140    150  \
0 2020-05-18 00:00:00  82065  15172  1500.0  3294  1086.0   962.0  28931   
1 2020-05-18 01:00:00  51248   9840   813.0  2356   696.0   546.0  17888   
2 2020-05-18 02:00:00  39026   7894   760.0  2413   408.0   549.0  13357   
3 2020-05-18 03:00:00  40993  10137   780.0  2701   420.0   741.0  15544   
4 2020-05-18 04:00:00  77863  19603  1276.0  5019   968.0  1160.0  32101   

    160   200   201    251   270   300   301   351    352   370   400    450  \
0  1103  3014  1619  12596  3060  7383  2958  5829  11297  2890  5694  19553   
1   720  1839  1502   7741  2132  4790  1743  4296   6781  1752  3651  12454   
2   498  1360  1233   6475  2098  3965  1345  3957   4762  1250  3199   9640   
3   532  1671  1800   7720  2133  4083  1501  3992   6202  1505  4459  10802   
4   968  2574  2384  12018  3502  6430  2882  5957  11987  3176  9054  16490   

     500    550   600   650   652   1000  1020  1040  1100  12

In [10]:
submission

       timestamp   10  100  101  120  121  140  150  160  200  201  251  270  \
0     20200525_0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1     20200525_1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2     20200525_2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3     20200525_3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4     20200525_4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
..           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
163  20200531_19  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
164  20200531_20  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
165  20200531_21  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
166  20200531_22  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
167  20200531_23  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

     300  301  351  352  370  400  450 

In [11]:
# train + valid
new_train = pd.concat([train, valid[168:]])
new_train

                     날짜        10      100      101      120     121     140  \
0   2020-01-01 00:00:00   83247.0  19128.0   2611.0   5161.0  1588.0   892.0   
1   2020-01-01 01:00:00   89309.0  19027.0   3337.0   5502.0  1650.0  1043.0   
2   2020-01-01 02:00:00   66611.0  14710.0   2970.0   4631.0  1044.0   921.0   
3   2020-01-01 03:00:00   53290.0  13753.0   2270.0   4242.0  1021.0   790.0   
4   2020-01-01 04:00:00   52095.0  17615.0   2406.0   3689.0  1840.0   922.0   
..                  ...       ...      ...      ...      ...     ...     ...   
331 2020-05-24 19:00:00  314226.0  98345.0  10625.0  28618.0  8316.0  6684.0   
332 2020-05-24 20:00:00  300001.0  87871.0   8226.0  22706.0  6981.0  5743.0   
333 2020-05-24 21:00:00  304150.0  71126.0   6002.0  18317.0  4939.0  3779.0   
334 2020-05-24 22:00:00  236751.0  44947.0   3575.0  11455.0  3135.0  2536.0   
335 2020-05-24 23:00:00  143609.0  26137.0   2242.0   6166.0  1609.0  1391.0   

          150     160      200     201 

In [12]:
new_train = pd.concat([train, valid[168:]])
new_train

                     날짜        10      100      101      120     121     140  \
0   2020-01-01 00:00:00   83247.0  19128.0   2611.0   5161.0  1588.0   892.0   
1   2020-01-01 01:00:00   89309.0  19027.0   3337.0   5502.0  1650.0  1043.0   
2   2020-01-01 02:00:00   66611.0  14710.0   2970.0   4631.0  1044.0   921.0   
3   2020-01-01 03:00:00   53290.0  13753.0   2270.0   4242.0  1021.0   790.0   
4   2020-01-01 04:00:00   52095.0  17615.0   2406.0   3689.0  1840.0   922.0   
..                  ...       ...      ...      ...      ...     ...     ...   
331 2020-05-24 19:00:00  314226.0  98345.0  10625.0  28618.0  8316.0  6684.0   
332 2020-05-24 20:00:00  300001.0  87871.0   8226.0  22706.0  6981.0  5743.0   
333 2020-05-24 21:00:00  304150.0  71126.0   6002.0  18317.0  4939.0  3779.0   
334 2020-05-24 22:00:00  236751.0  44947.0   3575.0  11455.0  3135.0  2536.0   
335 2020-05-24 23:00:00  143609.0  26137.0   2242.0   6166.0  1609.0  1391.0   

          150     160      200     201 

In [13]:
new_train = pd.concat([train, valid[168:]])
new_train.head()

                   날짜       10      100     101     120     121     140  \
0 2020-01-01 00:00:00  83247.0  19128.0  2611.0  5161.0  1588.0   892.0   
1 2020-01-01 01:00:00  89309.0  19027.0  3337.0  5502.0  1650.0  1043.0   
2 2020-01-01 02:00:00  66611.0  14710.0  2970.0  4631.0  1044.0   921.0   
3 2020-01-01 03:00:00  53290.0  13753.0  2270.0  4242.0  1021.0   790.0   
4 2020-01-01 04:00:00  52095.0  17615.0  2406.0  3689.0  1840.0   922.0   

       150     160      200     201      251     270      300     301     351  \
0  32263.0  1636.0   7938.0  1894.0  14957.0  3527.0  11174.0  2930.0  8640.0   
1  35609.0  1644.0   9896.0  1967.0  14682.0  3420.0  13884.0  3062.0  8349.0   
2  26821.0  1104.0   9343.0  1838.0  11120.0  3587.0  17141.0  3255.0  7025.0   
3  21322.0   909.0  10009.0  1630.0   9836.0  3842.0  20715.0  2661.0  7226.0   
4  22711.0  1354.0  15497.0  1264.0  10668.0  4559.0  28308.0  2894.0  8201.0   

       352     370     400      450      500      550      600

In [14]:
new_train = pd.concat([train, valid[168:]])
new_train.tail()

                     날짜        10      100      101      120     121     140  \
331 2020-05-24 19:00:00  314226.0  98345.0  10625.0  28618.0  8316.0  6684.0   
332 2020-05-24 20:00:00  300001.0  87871.0   8226.0  22706.0  6981.0  5743.0   
333 2020-05-24 21:00:00  304150.0  71126.0   6002.0  18317.0  4939.0  3779.0   
334 2020-05-24 22:00:00  236751.0  44947.0   3575.0  11455.0  3135.0  2536.0   
335 2020-05-24 23:00:00  143609.0  26137.0   2242.0   6166.0  1609.0  1391.0   

          150     160      200     201      251      270      300      301  \
331  141675.0  6619.0  23535.0  8572.0  67748.0  16791.0  33325.0  19124.0   
332  142933.0  6295.0  20953.0  7851.0  61609.0  14119.0  30216.0  18610.0   
333  133110.0  4781.0  15242.0  6101.0  48336.0  11534.0  25710.0  15651.0   
334   98582.0  3267.0   8849.0  4553.0  33607.0   8069.0  18293.0   9723.0   
335   54633.0  1899.0   5182.0  2422.0  20343.0   5104.0  11331.0   5413.0   

         351      352      370      400      450  

In [15]:
new_train

                     날짜        10      100      101      120     121     140  \
0   2020-01-01 00:00:00   83247.0  19128.0   2611.0   5161.0  1588.0   892.0   
1   2020-01-01 01:00:00   89309.0  19027.0   3337.0   5502.0  1650.0  1043.0   
2   2020-01-01 02:00:00   66611.0  14710.0   2970.0   4631.0  1044.0   921.0   
3   2020-01-01 03:00:00   53290.0  13753.0   2270.0   4242.0  1021.0   790.0   
4   2020-01-01 04:00:00   52095.0  17615.0   2406.0   3689.0  1840.0   922.0   
..                  ...       ...      ...      ...      ...     ...     ...   
331 2020-05-24 19:00:00  314226.0  98345.0  10625.0  28618.0  8316.0  6684.0   
332 2020-05-24 20:00:00  300001.0  87871.0   8226.0  22706.0  6981.0  5743.0   
333 2020-05-24 21:00:00  304150.0  71126.0   6002.0  18317.0  4939.0  3779.0   
334 2020-05-24 22:00:00  236751.0  44947.0   3575.0  11455.0  3135.0  2536.0   
335 2020-05-24 23:00:00  143609.0  26137.0   2242.0   6166.0  1609.0  1391.0   

          150     160      200     201 

In [16]:
new_train.sum()

10      8.084463e+08
100     2.334008e+08
101     2.498295e+07
120     6.072151e+07
121     2.320340e+07
140     1.638828e+07
150     3.468841e+08
160     1.534875e+07
200     3.991463e+07
201     2.184572e+07
251     1.658647e+08
270     4.335454e+07
300     6.918029e+07
301     4.287005e+07
351     9.078815e+07
352     1.512508e+08
370     2.829656e+07
400     7.915625e+07
450     1.673313e+08
500     2.663589e+08
550     1.728434e+08
600     4.888724e+07
650     3.871080e+07
652     2.273921e+07
1000    3.859388e+08
1020    1.800264e+07
1040    3.824057e+07
1100    9.432077e+07
1200    4.865134e+07
1510    1.704667e+07
2510    5.120751e+07
3000    8.521900e+06
4510    4.240068e+07
5510    2.790702e+07
6000    4.885059e+07
dtype: float64

In [17]:
cluster_0 = [('101', '남해선(영암-순천)'), ('120', '광주대구선'), ('121', '무안광주선'), ('140', '고창담양선'), ('160', '울산선'), ('201', '익산장수선'), ('251', '호남선'), ('270', '순천완주선'), ('301', '당진대전선'), ('550', '중앙선'), ('652', '동해선(부산-포항)'), ('1020', '남해1지선'), ('2510', '호남지선')]
cluster_1 = [('10', '경부선'), ('100', '남해선(순천-부산)'), ('150', '서해안선'), ('352', '중부선'), ('370', '제2 중부선'), ('400', '평택제천선'), ('450', '중부내륙선'), ('500', '영동선'), ('1000', '서울외곽순환선'), ('1040', '남해2지선'), ('1100', '제2경인선'), ('1200', '경인선'), ('3000', '대전남부선'), ('4510', '중부내륙지선'), ('5510', '중앙선지선'), ('6000', '부산외곽선')]
cluster_2 = [('200', '대구포항선'), ('300', '청주영덕선'), ('351', '통영대전선'), ('600', '서울양양선'), ('650', '동해선(삼척-속초)'), ('1510', '서천공주선')]

In [18]:
import os
import warnings

import pandas as pd
from datetime import datetime

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import numpy as np
import matplotlib.pyplot as plt

import wandb

pd.set_option('display.max_row', 100)
pd.set_option('display.max_column', 37)
warnings.filterwarnings(action='ignore')

In [19]:
def create_timestamp(date):
    result = []
    
    for i in range(0, 24):
        result.append(date + ' ' + str(i).zfill(2) + ':00:00')
    
    return result

newyear = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-01-01')),
    'holiday': 'newyear'
})

seol = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-01-24') + create_timestamp('2020-01-25') + create_timestamp('2020-01-26')),
    'holiday': 'seol'
})

seol_alter = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-01-27')),
    'holiday': 'seol_alter'
})

day31 = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-03-01')),
    'holiday': 'day31'
})

election = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-04-15')),
    'holiday': 'election'
})

budda_day = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-04-30')),
    'holiday': 'budda_day'
})

children_day = pd.DataFrame({
    'ds': pd.to_datetime(create_timestamp('2020-05-05')),
    'holiday': 'children_day'
})

# memorial_day = pd.DataFrame({
#     'hoilday': 'memorial_day',
#     'ds': pd.to_datetime(create_timestamp('2020-06-06')),
# })

# independence_day = pd.DataFrame({
#     'hoilday': 'independence_day',
#     'ds': pd.to_datetime(create_timestamp('2020-08-15')),
# })

# temp_hoilday = pd.DataFrame({
#     'hoilday': 'temp_hoilday',
#     'ds': pd.to_datetime(create_timestamp('2020-08-17')),
# })

# thanksgiving_day = pd.DataFrame({
#     'hoilday': 'thanksgiving_day',
#     'ds': pd.to_datetime(create_timestamp('2020-09-30'), create_timestamp('2020-10-01'), create_timestamp('2020-10-02')),
# })

# foundation_day = pd.DataFrame({
#     'hoilday': 'foundation_day',
#     'ds': pd.to_datetime(create_timestamp('2020-10-03')),
# })

In [20]:
holidays = pd.concat((newyear, seol, seol_alter, day31, election, budda_day, children_day))

In [21]:
holidays.info()

In [22]:
# Prophet 객체 선언
m = Prophet(
    holidays=holidays
)

# model 학습
m.fit(new_df)

# 예측값 설정
future = m.make_future_dataframe(periods=168, freq='H')

# predict
forecast = m.predict(future)
fig = m.plot(forecast,
            figsize=(40, 15))

fig = m.plot_components(forecast)

In [23]:
cluster_0 = [('101', '남해선(영암-순천)'), ('120', '광주대구선'), ('121', '무안광주선'), ('140', '고창담양선'), ('160', '울산선'), ('201', '익산장수선'), ('251', '호남선'), ('270', '순천완주선'), ('301', '당진대전선'), ('550', '중앙선'), ('652', '동해선(부산-포항)'), ('1020', '남해1지선'), ('2510', '호남지선')]
cluster_1 = [('10', '경부선'), ('100', '남해선(순천-부산)'), ('150', '서해안선'), ('352', '중부선'), ('370', '제2 중부선'), ('400', '평택제천선'), ('450', '중부내륙선'), ('500', '영동선'), ('1000', '서울외곽순환선'), ('1040', '남해2지선'), ('1100', '제2경인선'), ('1200', '경인선'), ('3000', '대전남부선'), ('4510', '중부내륙지선'), ('5510', '중앙선지선'), ('6000', '부산외곽선')]
cluster_2 = [('200', '대구포항선'), ('300', '청주영덕선'), ('351', '통영대전선'), ('600', '서울양양선'), ('650', '동해선(삼척-속초)'), ('1510', '서천공주선')]

In [24]:
from fbprophet.diagnostics import cross_validation, performance_metrics
import itertools

# 남해선
road = '101'

new_df = pd.DataFrame({
    'ds':new_train['날짜'],
    'y':new_train[road]
})

In [25]:
name = 'cluster_0'

config = {
    changepoint : 0.12,
    seasonality : 10,
    weekly : 17,
    daily : 13,
    holiday : 10
    
}

wandb.init(
    config=config,
    project='Prophet',
    tags=['prophet'],
    group='cluster 0',
    name= name,
    notes='cluster 0',
    save_code=True,
)

In [26]:
name = 'cluster_0'

config = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 10
    
}

wandb.init(
    config=config,
    project='Prophet',
    tags=['prophet'],
    group='cluster 0',
    name= name,
    notes='cluster 0',
    save_code=True,
)

In [27]:
m = Prophet(changepoint_prior_scale=config[changepoint],
            seasonality_prior_scale=config[seasonality],
            weekly_seasonality=config[weekly],
            daily_seasonality=config[holiday])

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

In [28]:
m = Prophet(changepoint_prior_scale=config['changepoint'],
            seasonality_prior_scale=config['seasonality'],
            weekly_seasonality=config['weekly'],
            daily_seasonality=config['daily'],
            holidays_prior_scale=
           )

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

In [29]:
m = Prophet(changepoint_prior_scale=config['changepoint'],
            seasonality_prior_scale=config['seasonality'],
            weekly_seasonality=config['weekly'],
            daily_seasonality=config['daily'],
            holidays=holidays,
            holidays_prior_scale=config['holiday']
           )

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

In [30]:
print(df_p['rmse'].mean())

wandb.log({
    'rmse':df_p['rmse'].mean()
})
wandb.finish()

In [31]:
new_train.sum()

10      8.084463e+08
100     2.334008e+08
101     2.498295e+07
120     6.072151e+07
121     2.320340e+07
140     1.638828e+07
150     3.468841e+08
160     1.534875e+07
200     3.991463e+07
201     2.184572e+07
251     1.658647e+08
270     4.335454e+07
300     6.918029e+07
301     4.287005e+07
351     9.078815e+07
352     1.512508e+08
370     2.829656e+07
400     7.915625e+07
450     1.673313e+08
500     2.663589e+08
550     1.728434e+08
600     4.888724e+07
650     3.871080e+07
652     2.273921e+07
1000    3.859388e+08
1020    1.800264e+07
1040    3.824057e+07
1100    9.432077e+07
1200    4.865134e+07
1510    1.704667e+07
2510    5.120751e+07
3000    8.521900e+06
4510    4.240068e+07
5510    2.790702e+07
6000    4.885059e+07
dtype: float64

In [32]:
num_list = [x[0] for x in cluster_0
num_list

In [33]:
num_list = [x[0] for x in cluster_0]
num_list

['101',
 '120',
 '121',
 '140',
 '160',
 '201',
 '251',
 '270',
 '301',
 '550',
 '652',
 '1020',
 '2510']

In [34]:
num_list = [x[0] for x in cluster_0]
new_train[num_list]

         101      120     121     140     160     201      251      270  \
0     2611.0   5161.0  1588.0   892.0  1636.0  1894.0  14957.0   3527.0   
1     3337.0   5502.0  1650.0  1043.0  1644.0  1967.0  14682.0   3420.0   
2     2970.0   4631.0  1044.0   921.0  1104.0  1838.0  11120.0   3587.0   
3     2270.0   4242.0  1021.0   790.0   909.0  1630.0   9836.0   3842.0   
4     2406.0   3689.0  1840.0   922.0  1354.0  1264.0  10668.0   4559.0   
..       ...      ...     ...     ...     ...     ...      ...      ...   
331  10625.0  28618.0  8316.0  6684.0  6619.0  8572.0  67748.0  16791.0   
332   8226.0  22706.0  6981.0  5743.0  6295.0  7851.0  61609.0  14119.0   
333   6002.0  18317.0  4939.0  3779.0  4781.0  6101.0  48336.0  11534.0   
334   3575.0  11455.0  3135.0  2536.0  3267.0  4553.0  33607.0   8069.0   
335   2242.0   6166.0  1609.0  1391.0  1899.0  2422.0  20343.0   5104.0   

         301      550      652    1020     2510  
0     2930.0  15393.0   2790.0  1311.0   3810.0  

In [35]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum()

101     2.498295e+07
120     6.072151e+07
121     2.320340e+07
140     1.638828e+07
160     1.534875e+07
201     2.184572e+07
251     1.658647e+08
270     4.335454e+07
301     4.287005e+07
550     1.728434e+08
652     2.273921e+07
1020    1.800264e+07
2510    5.120751e+07
dtype: float64

In [36]:
num_list = [x[0] for x in cluster_0]
max(new_train[num_list].sum())

172843406.1666667

In [37]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum()

101     2.498295e+07
120     6.072151e+07
121     2.320340e+07
140     1.638828e+07
160     1.534875e+07
201     2.184572e+07
251     1.658647e+08
270     4.335454e+07
301     4.287005e+07
550     1.728434e+08
652     2.273921e+07
1020    1.800264e+07
2510    5.120751e+07
dtype: float64

In [38]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum().max()

172843406.1666667

In [39]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum().max().index()

In [40]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum().idxmax()

'550'

In [41]:
cluster_0 = [('101', '남해선(영암-순천)'), ('120', '광주대구선'), ('121', '무안광주선'), ('140', '고창담양선'), ('160', '울산선'), ('201', '익산장수선'), ('251', '호남선'), ('270', '순천완주선'), ('301', '당진대전선'), ('550', '중앙선'), ('652', '동해선(부산-포항)'), ('1020', '남해1지선'), ('2510', '호남지선')]
cluster_1 = [('10', '경부선'), ('100', '남해선(순천-부산)'), ('150', '서해안선'), ('352', '중부선'), ('370', '제2 중부선'), ('400', '평택제천선'), ('450', '중부내륙선'), ('500', '영동선'), ('1000', '서울외곽순환선'), ('1040', '남해2지선'), ('1100', '제2경인선'), ('1200', '경인선'), ('3000', '대전남부선'), ('4510', '중부내륙지선'), ('5510', '중앙선지선'), ('6000', '부산외곽선')]
cluster_2 = [('200', '대구포항선'), ('300', '청주영덕선'), ('351', '통영대전선'), ('600', '서울양양선'), ('650', '동해선(삼척-속초)'), ('1510', '서천공주선')]

In [42]:
num_list = [x[0] for x in cluster_0]
new_train[num_list].sum().idxmax()

'550'

In [43]:
from fbprophet.diagnostics import cross_validation, performance_metrics
import itertools

num_list = [x[0] for x in cluster_0]
road = new_train[num_list].sum().idxmax()

# 중앙선
# road = '550'

new_df = pd.DataFrame({
    'ds':new_train['날짜'],
    'y':new_train[road]
})

In [44]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 10
    
}

wandb.init(
    config=cluster_0_params,
    project='Prophet',
    tags=['prophet'],
    group='cluster 0',
    name= name,
    notes='중앙선',
    save_code=True,
)

In [45]:
m = Prophet(changepoint_prior_scale=config['changepoint'],
            seasonality_prior_scale=config['seasonality'],
            weekly_seasonality=config['weekly'],
            daily_seasonality=config['daily'],
            holidays=holidays,
            holidays_prior_scale=config['holiday']
           )

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

In [46]:
print(df_p['rmse'].mean())

wandb.log({
    'rmse':df_p['rmse'].mean()
})
wandb.finish()

In [47]:
from fbprophet.diagnostics import cross_validation, performance_metrics
import itertools

num_list = [x[0] for x in cluster_0]
road = new_train[num_list].sum().idxmax()

# 중앙선
# road = '550'

new_df = pd.DataFrame({
    'ds':new_train['날짜'],
    'y':new_train[road]
})

In [48]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 11
    
}

wandb.init(
    config=cluster_0_params,
    project='Prophet',
    tags=['prophet'],
    group='cluster 0',
    name= name,
    notes='중앙선',
    save_code=True,
)

In [49]:
m = Prophet(changepoint_prior_scale=config['changepoint'],
            seasonality_prior_scale=config['seasonality'],
            weekly_seasonality=config['weekly'],
            daily_seasonality=config['daily'],
            holidays=holidays,
            holidays_prior_scale=config['holiday']
           )

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

In [50]:
print(df_p['rmse'].mean())

wandb.log({
    'rmse':df_p['rmse'].mean()
})
wandb.finish()

In [51]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 12
    
}

wandb.init(
    config=cluster_0_params,
    project='Prophet',
    tags=['prophet'],
    group='cluster 0',
    name= name,
    notes='중앙선',
    save_code=True,
)

In [52]:
m = Prophet(changepoint_prior_scale=config['changepoint'],
            seasonality_prior_scale=config['seasonality'],
            weekly_seasonality=config['weekly'],
            daily_seasonality=config['daily'],
            holidays=holidays,
            holidays_prior_scale=config['holiday']
           )

m.fit(new_df)
df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
df_p = performance_metrics(df_cv)

print(df_p['rmse'].mean())

wandb.log({
    'rmse':df_p['rmse'].mean()
})
wandb.finish()

In [53]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 10
    
}

In [54]:
cluster_0_params['holiday'] += 1

In [55]:
cluster_0_params

{'changepoint': 0.12,
 'seasonality': 10,
 'weekly': 17,
 'daily': 13,
 'holiday': 11}

In [56]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.12,
    'seasonality' : 10,
    'weekly' : 17,
    'daily' : 13,
    'holiday' : 10
    
}

In [57]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.05,
    'seasonality' : 10,
    'weekly' : 10,
    'daily' : 10,
    'holiday' : 10
    
}

In [58]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.05,
    'seasonality' : 10,
    'weekly' : 10,
    'daily' : 10,
    'holiday' : 10
    
}

In [59]:
for i in range(20):
    wandb.init(
        config=cluster_0_params,
        project='Prophet',
        tags=['holiday'],
        group='cluster 0',
        name= name,
        notes='중앙선',
        save_code=True,
    )

    m = Prophet(changepoint_prior_scale=cluster_0_params['changepoint'],
                seasonality_prior_scale=cluster_0_params['seasonality'],
                weekly_seasonality=cluster_0_params['weekly'],
                daily_seasonality=cluster_0_params['daily'],
                holidays=holidays,
                holidays_prior_scale=cluster_0_params['holiday']
               )

    m.fit(new_df)
    df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
    df_p = performance_metrics(df_cv)

    print(df_p['rmse'].mean())

    wandb.log({
        'rmse':df_p['rmse'].mean()
    })
    wandb.finish()
    cluster_0_params['holiday'] += 1

In [60]:
name = 'cluster_0'

cluster_0_params = {
    'changepoint' : 0.05,
    'seasonality' : 10,
    'weekly' : 10,
    'daily' : 10,
    'holiday' : 10
    
}

In [61]:
for i in range(20):
    wandb.init(
        config=cluster_0_params,
        project='Prophet',
        tags=['holiday'],
        group='cluster 0',
        name= name,
        notes='중앙선',
        save_code=True,
    )

    m = Prophet(changepoint_prior_scale=cluster_0_params['changepoint'],
                seasonality_prior_scale=cluster_0_params['seasonality'],
                weekly_seasonality=cluster_0_params['weekly'],
                daily_seasonality=cluster_0_params['daily'],
                holidays=holidays,
                holidays_prior_scale=cluster_0_params['holiday']
               )

    m.fit(new_df)
    df_cv = cross_validation(m, initial='14 days', period='7 days', horizon='7 days')
    df_p = performance_metrics(df_cv)

    print(df_p['rmse'].mean())

    wandb.log({
        'rmse':df_p['rmse'].mean()
    })
    wandb.finish()
    cluster_0_params['changepoint'] += 0.01